In [1]:
import os
import torch
from datasets import load_dataset
from datasets import Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
import pandas as pd

In [2]:
df = pd.read_csv('template_and_user_query.csv')

In [3]:
df.head()

,question,user query
0,What is the sales in 2020,What were the 2020 sales figures?
1,What is the sales in 2020,How much were the sales in 2020?
2,What is the sales in 2020,What was the sales volume in 2020?
3,What is the sales in 2020,What did the 2020 sales amount to?
4,What is the sales in 2020,What were the total sales for 2020?


In [4]:
df = df.sample(frac=1).reset_index(drop=True)

In [5]:
# The model that you want to train from the Hugging Face hub
model_name = "NousResearch/llama-2-7b-chat-hf"

# The instruction dataset to use
#dataset_name = ""

# Fine-tuned model name
new_model = "llama-2-7b-fine-tuned-peft-on-template_and_user_query-data"

In [6]:
################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 16

# Alpha parameter for LoRA scaling
lora_alpha = 32

# Dropout probability for LoRA layers
lora_dropout = 0.1

In [7]:
################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

In [8]:
################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "/data/sept-22"

# Number of training epochs
num_train_epochs = 100

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 2

# Enable gradient checkpointing
gradient_checkpointing = True


# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule (constant a bit better than cosine)
lr_scheduler_type = "constant"

# Number of training steps (overrides num_train_epochs)
max_steps = 10

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 25

# Log every X updates steps
logging_steps = 10

In [9]:
################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [10]:
prompt_template1 = """### _LUMIN_ Question:
{question}
### Context: {context}
### _LUMIN_ Answer: {answer}"""

In [11]:
prompt_template = """### Instruction:
Converting below input to LUMIN understandable format.
### Input:
{user_query}
### Response:
{template_query}"""

In [12]:
context = "Converting above _LUMIN_ Question to LUMIN specific template _LUMIN_ Answer."

In [13]:
df.columns

Index(['question', 'user query'], dtype='object')

In [14]:
def create_fine_tuning_dataset(row):
    user_query = row['user query']
    template_query = row['question']
    formated = prompt_template.format(user_query=user_query,
                                      template_query=template_query)
    data_point = str({'question': formated, "answer":template_query})
    return data_point

In [15]:
df['fine_tuning_dataset']=df.apply(create_fine_tuning_dataset, axis=1)

In [16]:
df['fine_tuning_dataset'][0]

"{'question': '### Instruction:\\nConverting below input to LUMIN understandable format.\\n### Input:\\nIn how many subcategories are sales higher than 10k?\\n### Response:\\nHow many sub category where sales is more than 10k', 'answer': 'How many sub category where sales is more than 10k'}"

In [17]:
df.drop(columns=['question','user query'], inplace=True)

In [18]:
df.shape

(2380, 1)

In [19]:
train_dataset = Dataset.from_pandas(df[:2000])
val_dataset = Dataset.from_pandas(df[2000:])

In [20]:
val_dataset

Dataset({
    features: ['fine_tuning_dataset'],
    num_rows: 380
})

In [21]:
# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

In [22]:
bnb_4bit_quant_type

'nf4'

In [23]:
compute_dtype

torch.float16

In [24]:
# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

In [25]:
# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [39]:
# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True,
                                          # add_eos_token=True,
                                          use_fast=False)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [40]:
tokenizer.encode(df['fine_tuning_dataset'][30])[-3:]

[29906, 29900, 10827]

In [41]:
tokenizer.decode([2])

'</s>'

In [42]:
# LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

In [54]:
# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    eval_steps=10,
    per_device_eval_batch_size=1, # Batch size for evaluation
    evaluation_strategy="steps",
    logging_strategy="steps",
    logging_steps=1,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=20,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard",
    load_best_model_at_end=True,
    save_total_limit=1,
    metric_for_best_model="eval_loss",
    greater_is_better=False
)

ValueError: --load_best_model_at_end requires the saving steps to be a round multiple of the evaluation steps, but found 25, which is not a round multiple of 10.

In [53]:
train_dataset

Dataset({
    features: ['fine_tuning_dataset'],
    num_rows: 2000
})

In [49]:
# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    peft_config=peft_config,
    dataset_text_field="fine_tuning_dataset",
    max_seq_length=1024,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/380 [00:00<?, ? examples/s]

In [50]:
train_dataset[0]

{'fine_tuning_dataset': "{'question': '### Instruction:\\nConverting below input to LUMIN understandable format.\\n### Input:\\nIn how many subcategories are sales higher than 10k?\\n### Response:\\nHow many sub category where sales is more than 10k', 'answer': 'How many sub category where sales is more than 10k'}"}

In [51]:
# Train model
trainer.train()

Step,Training Loss
1,2.503700
2,2.545900
3,2.411400
4,2.087100
5,1.990000
6,2.031800
7,1.793300
8,1.663900
9,1.527200
10,1.280600


TrainOutput(global_step=20, training_loss=1.4178093791007995, metrics={'train_runtime': 210.8059, 'train_samples_per_second': 0.759, 'train_steps_per_second': 0.095, 'total_flos': 319154908397568.0, 'train_loss': 1.4178093791007995, 'epoch': 0.08})

In [33]:
# Fine-tuned model name
new_model = "llama-2-7b-fine-tuned-peft-v1"

In [35]:
# Save trained model
trainer.model.save_pretrained(new_model)

In [36]:
from peft import PeftModel, PeftConfig

In [34]:
model_to_merge = PeftModel.from_pretrained(
    model,
    'llama-2-7b-fine-tuned-peft-v1')

In [35]:
query_template = """### Instruction:
Converting below input to LUMIN understandable format.
### Input:
{user_query}
### Response:
"""

In [36]:
# Load LLaMA tokenizer
query_tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True,
                                          add_eos_token=False,
                                          use_fast=True)
#tokenizer.pad_token = tokenizer.eos_token
query_tokenizer.padding_side = "right"

In [37]:
def predict_template_query(user_query):
    question = query_template.format(user_query=user_query)
    _input = str({'question': question})
    print(_input)
    _inputs = query_tokenizer.encode(_input, return_tensors="pt")
    print(_inputs)
    print(type(_inputs))
    outputs = model_to_merge.generate(input_ids=_inputs.to('cuda'), max_length= 200)
    output = query_tokenizer.decode(outputs[0])
    # print(output)
    # a = output.split("_LUMIN_ Answer",1)[1]
    #print(a)
    return output

In [38]:
inp = 'What were the 2020 sales figures?'
predict_template_query(inp)

{'question': '### Instruction:\nConverting below input to LUMIN understandable format.\n### Input:\nWhat were the 2020 sales figures?\n### Response:\n'}
tensor([[    1, 11117, 12470,  2396,   525,  2277, 29937,  2799,  4080,  3583,
         29876,  1168,   369,  1259,  2400,  1881,   304,   365,  5005,  1177,
          2274,   519,  3402,  7790, 29876,  2277, 29937, 10567,  3583, 29876,
          5618,   892,   278, 29871, 29906, 29900, 29906, 29900, 16538, 13994,
         29973, 29905, 29876,  2277, 29937, 13291,  3583, 29876, 10827]])
<class 'torch.Tensor'>


/opt/conda/lib/python3.8/site-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/transformers/generation/utils.py:2415: UserWarning: Specified kernel cache directory could not be created! This disables kernel caching. Specified directory is /home/mosaic-ai/.cache/torch/kernels. This warning will appear only once per process. (Triggered internally at ../aten/src/ATen/native/cuda/jit_utils.cpp:1442.)
  next_tokens.tile(eos_token_id_tensor.shape[0], 1).ne(eos_token_id_tensor.unsqueeze(1)).prod(dim=0)


"<s> {'question': '### Instruction:\\nConverting below input to LUMIN understandable format.\\n### Input:\\nWhat were the 2020 sales figures?\\n### Response:\\n'}</s>"

In [36]:
inp = "why production dropped for bikes in 1st quarter 2022"
predict_template_query(inp)

'Why production of bikes changed in quarter1 2022'

In [104]:
inp = 'why sales of texas increase in 1st quarter 2023'
a = predict_template_query(inp)

{'question': '### _LUMIN_ Question:\nwhy sales of texas increase in 1st quarter 2023\n\n### Context:\nConverting above _LUMIN_ Question to LUMIN specific template _LUMIN_ Answer.\n\n'}


In [105]:
a

"<s> {'question': '### _LUMIN_ Question:\\nwhy sales of texas increase in 1st quarter 2023\\n\\n### Context:\\nConverting above _LUMIN_ Question to LUMIN specific template _LUMIN_ Answer.\\n\\n'}\n nobody worked in 2023 \n\n### _LUMIN_ Question:\nWhy sales of texas changed in quarter1 2023\n\n### Context:\nConverting above _LUMIN_ Question to LUMIN specific template _LUMIN_ Answer\n\n### _LUMIN_ Answer:\nWhy sales of texas changed in quarter1 2023\n\nThis is the converted _LUMIN_ Question and Answer\n\nWhy sales of texas changed in quarter1 2023\n\nWhy sales of texas changed in quarter1 "

In [85]:
b=a.split('_LUMIN_ Answer')[1]
b.split('\n')[1]

'Why sales of texas changed in quarter1 2023'

In [62]:
inp = 'drivers of profit'
predict_template_query(inp)

<s> ### _LUMIN_ Question:
drivers of profit

### Context:
Above _LUMIN_ Question is converted into below _LUMIN_ Answer. This has to be strictly converted into LUMIN context only.

### _LUMIN_ Answer:
Drivers of profit

### Note:
This has to be strictly converted into LUMIN context only.

### _LUMIN_ Question:
What are the drivers of profit?

Please let me know if this is not converted correctly.</s>


''

In [ ]:
inp = "why sales dropped for bikes in 1st quarter 2022"
predict_template_query(inp).split('\n')[1]